# Trusted zone
Joins tables per data source

In [346]:
# ! pip install duckdb --user --upgrade --quiet
# ! pip install pyarrow --user --upgrade --quiet

In [347]:
import os
import duckdb
import numpy as np
import pandas as pd

In [348]:
def df_to_DBtable(DB,df,table): # repeated
    """
    Creates a persistent table in DuckDB from the DataFrame content.
    """
    con = duckdb.connect(DB)
    con.register(table, df)
    con.execute(f'CREATE TABLE {table} AS SELECT * FROM {table}')
    con.close()

def DBtable_to_df(table):
    """
    Converts the DB `table` in a data frame format 
    """
    repo = ''.join(filter(str.isalpha,table))
    con = duckdb.connect(f'{repo}.duckdb')
    df = con.execute(f'SELECT * FROM {table}').df()
    con.close()
    return df

def get_tables(DB):
    """
    Gets all the tables from the `DB`
    """
    con = duckdb.connect(DB)
    tables = con.execute(f'SELECT table_name FROM information_schema.tables').df()['table_name']
    con.close()
    return tables

def pre_join(DB):
    """
    Adds a version (year) column to all `DB` tables in order to do the joining.
    Returns all the `DB` tables in data frame format.
    """
    tablesDB = get_tables(DB)
    dfs = []
    for table in tablesDB:
        year = ''.join(filter(str.isnumeric,table))
        df = DBtable_to_df(table)
        # display(df)
        df['Year'] = year
        dfs.append(df)
    return dfs

def trusted_zone(DB,table):
    """
    Joins all the `DB` tables into one and stores it in the `DB`.
    """
    dfs = pre_join(DB)
    df = pd.concat(dfs, axis=0, ignore_index=True)
    # display(df)
    df_to_DBtable(DB,df,table)

In [349]:
trusted_zone('nationalities.duckdb', 'nationalities')
trusted_zone('household.duckdb', 'household')

### Test

#### nationalities 

In [350]:
# nationalities joined
con = duckdb.connect('nationalities.duckdb')
view_nationalities = con.execute(f'SELECT * FROM nationalities').df()
display(view_nationalities)
con.close()

,Madrid_section,Habitantes,Españoles,Extranjeros,Alemania,Austria,Bélgica,Bulgaria,Chipre,Croacia,...,Samoa,Timor_Oriental,Otros_Países_de_Oceanía,Year,Bielorrusia,Djibouti,Surinam,Islas_Marshall,Vanuatu,República_Democrática_del_Congo
0,0014,84.0,83.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2019,NaN,NaN,NaN,NaN,NaN,NaN
1,001401,84.0,83.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2019,NaN,NaN,NaN,NaN,NaN,NaN
2,001401001,84.0,83.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2019,NaN,NaN,NaN,NaN,NaN,NaN
3,0029,4712.0,4098.0,614.0,8.0,0.0,1.0,9.0,0.0,0.0,...,0.0,0.0,0.0,2019,NaN,NaN,NaN,NaN,NaN,NaN
4,002901,4712.0,4098.0,614.0,8.0,0.0,1.0,9.0,0.0,0.0,...,0.0,0.0,0.0,2019,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14518,182101,262.0,235.0,27.0,0.0,0.0,0.0,4.0,0.0,0.0,...,NaN,0.0,0.0,2020,NaN,0.0,NaN,NaN,0.0,0.0
14519,182101001,262.0,235.0,27.0,0.0,0.0,0.0,4.0,0.0,0.0,...,NaN,0.0,0.0,2020,NaN,0.0,NaN,NaN,0.0,0.0
14520,1837,1734.0,1475.0,259.0,1.0,0.0,0.0,1.0,0.0,0.0,...,NaN,0.0,0.0,2020,NaN,0.0,NaN,NaN,0.0,0.0
14521,183701,1734.0,1475.0,259.0,1.0,0.0,0.0,1.0,0.0,0.0,...,NaN,0.0,0.0,2020,NaN,0.0,NaN,NaN,0.0,0.0


#### household

In [351]:
# households joined
con = duckdb.connect('household.duckdb')
view_household = con.execute(f'SELECT * FROM household').df()
display(view_household)
con.close()

,section,population,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors,three_adults_and_0_or_more_minors,two_adults_over_35_and_two_adults_from_16_to_34,two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor,two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors,four_adults_and_0_or_more_minors,five_adults_and_0_or_more_minors,fifteen_or_more_inhabitants,only_minors,Year
0,Ciudad de Madrid,1278258.0,116921.0,114806.0,124815.0,35963.0,27460.0,5488.0,156118.0,173809.0,...,8792.0,102020.0,50724.0,7803.0,3361.0,45696.0,50463.0,513.0,342.0,2018
1,01. Centro,66411.0,11695.0,13210.0,5173.0,2142.0,970.0,191.0,12277.0,5164.0,...,191.0,4013.0,1060.0,183.0,79.0,1718.0,1854.0,43.0,18.0,2018
2,1001.0,542.0,72.0,86.0,63.0,16.0,10.0,4.0,82.0,63.0,...,0.0,30.0,13.0,2.0,2.0,13.0,18.0,0.0,0.0,2018
3,1002.0,481.0,100.0,91.0,36.0,19.0,7.0,0.0,89.0,28.0,...,1.0,38.0,9.0,1.0,1.0,10.0,11.0,0.0,1.0,2018
4,1003.0,884.0,154.0,179.0,74.0,40.0,13.0,2.0,141.0,77.0,...,3.0,54.0,18.0,4.0,0.0,23.0,30.0,0.0,1.0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,21029.0,514.0,29.0,26.0,6.0,7.0,17.0,7.0,41.0,23.0,...,10.0,25.0,49.0,10.0,0.0,14.0,18.0,0.0,0.0,2020
7391,21030.0,561.0,35.0,59.0,27.0,8.0,17.0,5.0,61.0,50.0,...,4.0,44.0,43.0,5.0,0.0,13.0,17.0,0.0,0.0,2020
7392,21031.0,988.0,106.0,96.0,17.0,9.0,37.0,7.0,156.0,40.0,...,7.0,33.0,32.0,7.0,1.0,15.0,28.0,1.0,0.0,2020
7393,21032.0,683.0,71.0,65.0,16.0,8.0,31.0,17.0,105.0,27.0,...,7.0,23.0,30.0,6.0,0.0,8.0,7.0,1.0,0.0,2020
